In [ ]:
import nltk
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from sklearn.model_selection import GridSearchCV
from sklearn import svm
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score

import string
import pickle
import pandas as pd
nltk.download('all')
# initialize the stemmer
ps = PorterStemmer()

# define the functions
def get_importantFeatures(sent):
    sent_str = str(sent)
    returnList = []
    sent_tokens = nltk.word_tokenize(sent_str)
    for token in sent_tokens:
        if token.isalnum():
            returnList.append(token)
    return returnList

def removing_stopWords(sent):
    returnList = []
    for i in sent:
        if i not in stopwords.words('english') and i not in string.punctuation:
            returnList.append(i)
    return returnList

def potter_stem(sent):
    returnList = []
    for i in sent:
        returnList.append(ps.stem(i))
    return " ".join(returnList)

# create the dataframe
df = pd.read_csv('/spamCT.csv')


# apply the functions to the dataframe
df['imp_feature'] = df['EmailText'].apply(get_importantFeatures)
df['imp_feature'] = df['imp_feature'].apply(removing_stopWords)
df['imp_feature'] = df['imp_feature'].apply(potter_stem)
from sklearn.model_selection import train_test_split
X = df['imp_feature']
y = df['Label']
# split the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)
tfidf = TfidfVectorizer()

# convert X_test to a list of strings
X_train=X_train.apply(str)
y_train=y_train.apply(str)
X_test=X_test.apply(str)
y_test=y_test.apply(str)

# create a TfidfVectorizer object and fit it to the training data
feature = tfidf.fit_transform(X_train)

# specify the hyperparameters to tune
tuned_parameters = {'kernel': ['linear', 'rbf'], 'gamma': [1e-3, 1e-4], 'C': [1, 10, 100, 1000],'probability':'True'}

# create a GridSearchCV object and fit it to the training data
model = GridSearchCV(svm.SVC(), tuned_parameters)
model.fit(feature, y_train)

# transform the test data using the fitted TfidfVectorizer object
y_predict = tfidf.transform(X_test)
print("Accuracy:", model.score(y_predict, y_test))

In [ ]:
from sklearn.metrics import roc_auc_score
y_prob = model.predict_proba(tfidf.transform(X_test))[:, 1]
roc_auc = roc_auc_score(y_test, y_prob)
print("ROC AUC Score:", roc_auc)
